In [ ]:
import pandas as pd
import seaborn as sns
from warnings import filterwarnings as fw
fw('ignore')
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics as skm
import tkinter as tk
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv("/kaggle/input/predict-restaurant-revenue/train.csv")

In [ ]:
df.head(15)


In [ ]:
df.info()
##Getting all the  null values

In [ ]:
df['City'].nunique()

In [ ]:
df = df.astype({"City": 'category', "City Group":'category', "Type":'category'})
# ##converting dtype to category for encoding

In [ ]:
#mapping the cateogorical data type.
df['City Group'] = df[['City Group']].apply(lambda x: x.cat.codes)
df['Type'] = df[['Type']].apply(lambda x: x.cat.codes)
df['City'] = df[['City']].apply(lambda x: x.cat.codes)

In [ ]:
df.head()

In [ ]:
# Extracting year month and date
df["Year"] = pd.DatetimeIndex(df["Open Date"]).year
df["Month"] = pd.DatetimeIndex(df["Open Date"]).month
df["Day"] = pd.DatetimeIndex(df["Open Date"]).day

df.head()

In [ ]:
# plot of correlation of columns with revenue

fig = plt.figure(figsize=(20,16))
target_corr = df[df.columns[2:]].corr()['revenue']
order_corr = target_corr.sort_values()
y = pd.DataFrame(order_corr).index[:-1]
x = pd.DataFrame(order_corr).revenue[:-1]
sns.barplot(x, y, orient='h')
plt.show()

In [ ]:
# checking linearity
sns.pairplot(df,x_vars = df.columns.drop(['Id', 'Open Date']), y_vars='revenue', height=7, aspect=0.7, kind='reg')

In [ ]:
df

In [ ]:
##linear regresssion
x = df[['Year', 'City Group', 'City', 'Type']]
y = df["revenue"]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state = 7)

reg = LinearRegression().fit(xtrain, ytrain)
y_pred = reg.predict(xtest)

mae = skm.mean_absolute_error(ytest, y_pred)*len(ytest)
p1 = mae/ytest.sum()
accuracy_linear = (100 - (p1*100))
print(accuracy_linear)

reg.score(xtest, ytest)*100

In [ ]:
final_model = reg
pd.to_pickle(final_model, 'RestaurantRevenuePredictor')

In [ ]:
# predection model
model = pd.read_pickle('RestaurantRevenuePredictor')

# mapping city values to convert categorical data into numerical data
df = pd.read_csv('/kaggle/input/predict-restaurant-revenue/train.csv')
city_labels = df['City'].astype('category').cat.categories.tolist()
map_city = {'City' : {i : v for i,v in zip(city_labels,list(range(0,len(city_labels))))}}

# gui
app = tk.Tk()
app.title('Restaurant Revenue Prediction')
app.geometry('400x300')

# variables
revenue_var = tk.IntVar() 
revenue_var.set(0)
year_var = tk.Variable(app)           
city_clicked = tk.Variable()
city_group_clicked = tk.Variable()
type_clicked = tk.Variable()

# creating labels
tk.Label(app, text = 'Opening year of restaurant : ').place(x = 25, y = 25)
tk.Label(app, text = 'City in which restaurant is located : ').place(x = 25, y = 70 )
tk.Label(app, text = 'City Group : ').place(x = 25 , y = 115)
tk.Label(app, text = 'Type : ').place(x = 25 , y = 160)
tk.Label(app, textvariable = revenue_var).pack(side = 'bottom')

# creating entry tab
tk.Entry(app, textvariable = year_var, width = 10).place(x = 250, y = 25)

# drop-down menu
city_options = list(map_city['City'].keys())
city_group_options = ['Big Cities', 'Other']
type_options = ['Food Court', 'Inline', 'Drive Thru']

# drop-down menu default value
city_clicked.set("City")
city_group_clicked.set("City Group")
type_clicked.set("Type")

# drop-down
city_drop = tk.OptionMenu( app , city_clicked , *city_options ).place(x = 250, y = 70)
city_group_drop = tk.OptionMenu( app , city_group_clicked , *city_group_options ).place(x = 250, y = 115)
type_drop = tk.OptionMenu( app , type_clicked , *type_options ).place(x = 250, y = 160)

# function for prediction
def predict():
    try:
        if(opening_year >= 1800 and opening_year < 2000):
            opening_year = year_var.get()
        city = city_clicked.get()
        city_group = city_group_clicked.get()
        type_ = type_clicked.get()
    
#       convert 'City Group' to numerical data
        if(city_group == 'Other'):
            city_group = 0
        elif(city_group == 'Big Cities'):
            city_group = 1
        else:
            city_group = None
            
    
#       convert 'type' intpo numerical data
        if(type_ == 'Food Court'):
            type_ = 1
        elif(type_ == 'Inline'):
            type_ = 2
        elif(type_ == 'Drive Thru'):
            type_ = 0
        else:
            type_ = None
        
#         creating database for revenue prediction 
        query = pd.DataFrame({'Year': [opening_year], 
                              'City': map_city['City'][city],
                              'City Group': [city_group],
                              'Type': [type_]})
    
#       setting predicted value for display
        revenue_var.set(model.predict(query)[0])
    
    except Exception:
        print('Incomplete Input')
        

# button
tk.Button(app, text = 'Predict Revenue', command = predict).place(x = 150, y = 230)


app.mainloop()